<a href="https://colab.research.google.com/github/23Aditya/TransferLearning-in-Keras/blob/main/End_to_end_Transfer_Learning_keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from keras.applications import vgg16
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense,Flatten,BatchNormalization
from keras.activations import relu
from keras.initializers import glorot_uniform
from keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from keras.models import Sequential

In [ ]:
data_gen = ImageDataGenerator()

In [ ]:
import os

In [ ]:
training_datagen = data_gen.flow_from_directory(directory="DevanagariHandwrittenCharacterDataset/Train",
                                                     target_size=(32,32),color_mode="rgb",
                                                     classes=os.listdir("DevanagariHandwrittenCharacterDataset/Train"),
                                                     batch_size=50)

Found 62560 images belonging to 46 classes.


In [ ]:
cv_datagen = data_gen.flow_from_directory(directory="DevanagariHandwrittenCharacterDataset/cv",
                                                    target_size=(32,32),color_mode="rgb",
                                                    classes=os.listdir("DevanagariHandwrittenCharacterDataset/cv"),
                                                    batch_size=15640)

Found 15640 images belonging to 46 classes.


In [ ]:
testing_datagen = data_gen.flow_from_directory(directory="DevanagariHandwrittenCharacterDataset/Test",
                                                   target_size=(32,32),color_mode="rgb",
                                                   classes=os.listdir("DevanagariHandwrittenCharacterDataset/Test"),
                                                   batch_size=13800)

Found 13800 images belonging to 46 classes.


In [ ]:
conv_base = vgg16.VGG16(include_top=False,input_shape=(32,32,3))

In [ ]:
conv_base.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 32, 32, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 32, 32, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 16, 16, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 16, 16, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 16, 16, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 8, 8, 128)         0     

In [ ]:
conv_base.trainable = False

In [ ]:
conv_base.layers[::-1]

In [ ]:
def unfreeze_layer():
    trainable = False
    
    for layer in conv_base.layers:
        if layer.name == layer.name:
            trainable = True
        
        if trainable:
            layer.trainable=True
        else:
            layer.trainable=False
        
    return conv_base
    

In [ ]:
conv_base.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 32, 32, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 32, 32, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 16, 16, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 16, 16, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 16, 16, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 8, 8, 128)         0     

In [ ]:
def build_network():

  network = Sequential()
  conv_base = unfreeze_layer()
  network.add(conv_base)
  network.add(Flatten())
  network.add(Dense(units=512,activation="relu",kernel_initializer="glorot_normal"))
  network.add(BatchNormalization())
  network.add(Dense(units=46,activation="softmax",kernel_initializer="glorot_normal"))
  
  network.compile(optimizer=Adam(),loss="categorical_crossentropy",metrics=["accuracy"])

  return network

In [ ]:
network = build_network()

In [ ]:
network.summary()

In [ ]:
network.fit_generator(generator=training_datagen,epochs=2,validation_data=cv_datagen,
                      workers=2,use_multiprocessing=True)